In [ ]:
# ! pip install mnist
%matplotlib inline
import numpy as np
import mnist
import matplotlib.pyplot as plt

In [ ]:
def display_image(image):
    plt.imshow(image, cmap="gray")

def get_mnist_threes_nines():
    # 80/20 train test split
    Y0 = 3
    Y1 = 9

    y_train = mnist.train_labels()
    y_test = mnist.test_labels()
    X_train = (mnist.train_images()/255.0)
    X_test = (mnist.test_images()/255.0)
    train_idxs = np.logical_or(y_train == Y0, y_train == Y1)
    test_idxs = np.logical_or(y_test == Y0, y_test == Y1)
    y_train = y_train[train_idxs].astype('int')
    y_test = y_test[test_idxs].astype('int')
    X_train = X_train[train_idxs]
    X_test = X_test[test_idxs]
    y_train = (y_train == Y1).astype('int')
    y_test = (y_test == Y1).astype('int')
    return (X_train, y_train), (X_test, y_test)


In [ ]:
(X_train, y_train), (X_test, y_test) = get_mnist_threes_nines()
image = X_train[1]
display_image(image)
print("Label:", y_train[1])

In [ ]:
################################################
############ Naive Sigmoid #####################
################################################

def sigmoid_activation(s):
    out = 1 / (1 + np.exp(-s))
    grad = out * (1 - out)
    return out, grad

s = np.asarray([1., 0., -1])
out, grad = sigmoid_activation(s)
print(out)
print(grad)

In [ ]:
################################################
############ Better Sigmoid ####################
################################################
def sigmoid_activation(s):
    mask = (s >= 0)
    out = np.where(mask, 
                   1 / (1 + np.exp(-s, where=mask)), 
                   np.exp(s, where=~mask) / (1 + np.exp(s, where=~mask)))
    grad = out * (1 - out)
    return out, grad

In [ ]:
def sigmoid_activation(x):
    mask = (x >= 0)
    pos_sig = 1 / (1 + np.exp(-x, where=mask, out=np.ones_like(x)))
    neg_sig = np.exp(x, where=~mask, out=np.ones_like(x)) / \
        (1 + np.exp(x, where=~mask, out=np.ones_like(x)))
    out = np.where(mask, 
                   pos_sig, 
                   neg_sig)
    eps = 1e-15
    out = np.clip(out, eps, 1. - eps)
    grad = out * (1 - out)
    return out, grad


def logistic_loss(g, y):
    """
    Computes the loss and gradient for binary classification with logistic
    loss

    Inputs:
    - g: Output of final layer with sigmoid activation,
         of shape (N, 1) 

    - y: Vector of labels, of shape (N,) where y[i] is the label for x[i] 
         and y[i] in {0, 1}

    Returns a tuple of:
    - loss: array of losses
    - dL_dg: Gradient of the loss with respect to g
    """
    assert(g.ndim == 2 and g.shape[1] == 1)
    assert(y.ndim == 1)
    g = g.reshape(-1)
    loss = -(y * np.log(g) + (1 - y) * np.log(1 - g))
    dL_dg = -((y / g) + (y - 1)/(1 - g))
    return loss, dL_dg

In [ ]:
def relu_activation(s):
    mask = s <= 0
    out = np.where(mask, 0, s)
    ds = np.where(mask, 0.0, 1.0)
    return out, ds


In [ ]:
def layer_forward(x, W, b, activation_fn):
    assert(x.ndim == 2)
    assert(W.ndim == 2 and W.shape[0] == x.shape[1])
    assert(b.ndim == 2 and b.shape[0] == 1 and b.shape[1] == W.shape[1])
    h = x @ W + b
    out, ds = activation_fn(h)
    cache = (x, W, b, ds)
    return out, cache


In [ ]:
def create_weight_matrices(layer_dims):
    """
    Creates a list of weight matrices defining the weights of NN
    
    Inputs:
    - layer_dims: A list whose size is the number of layers. layer_dims[i] defines
      the number of neurons in the i+1 layer.

    Returns a list of weight matrices
    """
    weights = []
    for i in range(len(layer_dims) - 1):
        nrow = layer_dims[i]
        ncol = layer_dims[i+1]
        W = np.random.randn(nrow, ncol) * 0.01
        weights.append(W)
    return weights


def create_bias_vectors(layer_dims):
    """
    Creates a list of weight matrices defining the weights of NN
    
    Inputs:
    - layer_dims: A list whose size is the number of layers. layer_dims[i] defines
      the number of neurons in the i+1 layer.

    Returns a list of weight matrices
    """
    biases = [np.random.randn(1, h)* 0.01 for h in layer_dims[1:]]
    return biases

In [ ]:
def forward_pass(X_batch, weight_matrices, biases, activations):
    assert(len(weight_matrices) == len(biases))
    assert(len(weight_matrices) == len(activations))
    layer_caches = []
    h = X_batch
    for layer in range(len(weight_matrices)):
        W = weight_matrices[layer]
        b = biases[layer]
        activation_fn = activations[layer]
        h, layer_cache = layer_forward(h, W, b, activation_fn)
        layer_caches.append(layer_cache)
    output = h
    return output, layer_caches

In [ ]:
def backward_pass(dL_dg, layer_caches):

    grad_Ws = []
    grad_bs = []

    ##########################################################
    ################### Base Case ############################
    ##########################################################
    layer_caches_reversed = layer_caches[::-1]
    x_l_min_1, W_l, b_l, g_prime = layer_caches_reversed[0]
    delta_l = dL_dg.reshape(-1, 1) * g_prime

    grad_W = np.einsum("ni,nj->nij", x_l_min_1, delta_l).mean(axis=0)
    grad_b = delta_l.mean(axis=0, keepdims=True)

    grad_Ws.insert(0, grad_W)
    grad_bs.insert(0, grad_b)

    W_dot_delta = np.einsum("ij,nj->ni", W_l, delta_l)

    ##########################################################
    ################### Inductive Step #######################
    ##########################################################
    for layer in layer_caches_reversed[1:]:
        x_l_min_1, W_l, b_l, g_prime = layer
        delta_l = g_prime * W_dot_delta

        grad_W = np.einsum("ni,nj->nij", x_l_min_1, delta_l).mean(axis=0)
        grad_b = delta_l.mean(axis=0, keepdims=True)

        grad_Ws.insert(0, grad_W)
        grad_bs.insert(0, grad_b)

        W_dot_delta = np.einsum("ij,nj->ni", W_l, delta_l)
    return grad_Ws, grad_bs


Train Network

In [ ]:
layer_dims = [28*28, 200, 1]
activations = [relu_activation, sigmoid_activation]
weight_matrices = create_weight_matrices(layer_dims)
biases = create_bias_vectors(layer_dims)
bs = 100
step_size = 0.1
training_stats = []
for epoch in range(5):
    idxs = np.arange(X_train.shape[0])
    idxs = np.random.permutation(idxs)
    for batch in range(X_train.shape[0] // bs):
        batch_idxs = idxs[batch * bs : (batch + 1) * bs]
        X_batch = X_train[batch_idxs]
        X_batch = X_batch.reshape(X_batch.shape[0], -1)
        y_batch = y_train[batch_idxs]

        output, layer_caches = forward_pass(X_batch, weight_matrices, biases, activations)
        loss, dL_dg = logistic_loss(output, y_batch)
        grad_Ws, grad_bs = backward_pass(dL_dg, layer_caches)

        #######################################################
        ################ Update Gradients #####################
        #######################################################
        for weight_matrix, bias, grad_W, grad_b in zip(weight_matrices, biases, grad_Ws, grad_bs):
            weight_matrix -= (step_size * grad_W)
            bias -= (step_size * grad_b)

        y_hat = (output > 0.5).astype(np.int).reshape(-1)

        accuracy = (y_hat == y_batch).sum() / y_batch.shape[0]

        avg_train_loss = loss.mean()
        avg_train_acc = accuracy


        #######################################################
        ############### Test Loss #############################
        #######################################################
        output, _ = forward_pass(X_test.reshape(X_test.shape[0], -1), weight_matrices, biases, activations)
        loss, ds = logistic_loss(output, y_test)

        y_hat = (output > 0.5).astype(np.int).reshape(-1)
        accuracy = (y_hat == y_test).sum() / y_test.shape[0]

        avg_test_loss = loss.mean()
        avg_test_acc = accuracy

        training_stats.append([avg_train_loss, avg_train_acc, avg_test_loss, avg_test_acc])

        print(f"Test Loss: {loss.mean():.3f}")
        print(f"Test Accuracy: {accuracy:.3f}")



Look at loss and accuracy

In [ ]:
import pandas as pd
df = pd.DataFrame(training_stats, columns = ["Train_Loss", "Train_Acc", "Test_Loss", "Test_Acc"])
fig, ax = plt.subplots()
ax.plot(df.Train_Loss, label="Training Loss")
ax.plot(df.Test_Loss, label="Test Loss")
ax.set(ylim=(0, 1.))
ax.legend()
fig.tight_layout()
plt.savefig("train_test_losses_lr_01.png", dpi=300)

fig, ax = plt.subplots()
ax.plot(df.Train_Acc, label="Training Accuracy")
ax.plot(df.Test_Acc, label="Test Accuracy")
ax.legend()
fig.tight_layout()
plt.savefig("train_test_accuracy_lr_01.png", dpi=300)



Look at wrong predictions

In [ ]:
output, _ = forward_pass(X_test.reshape(X_test.shape[0], -1), weight_matrices, biases, activations)
loss, dL_dg = logistic_loss(output, y_test)

y_hat = (output > 0.5).astype(np.int).reshape(-1)

wrong_preds = X_test[y_hat != y_test]
wrong_preds_label = y_test[y_hat != y_test]



i = 0
display_image(wrong_preds[i]), wrong_preds_label[i]

Check gradient with numerical differences

In [ ]:
def fd_grad(func, input_array, eps=1e-5):
	"""
	`func` takes in a (flat!) array and returns a scalar
	`input_array` is assumed to be flat and the correct size for inputting
	"""
	input_array = input_array.copy()
	grad = np.zeros_like(input_array)
	for i in range(input_array.shape[0]):
	    input_array[i] += eps
	    val1 = func(input_array)
	    input_array[i] -= (2 * eps)
	    val2 = func(input_array)
	    input_array[i] += eps
	    grad[i] = (val1 - val2) / (2 * eps)
	return grad

def backward_pass_checker(X_batch, y_batch, weight_matrices, biases, activations):
    grad_Ws, grad_bs = [], []
    for i, W in enumerate(weight_matrices):
        def checker(W_flat):
            weight_matrices[i] = W_flat.reshape(W.shape)
            output, layer_caches = forward_pass(X_batch, weight_matrices, biases, activations)
            loss, dL_dg = logistic_loss(output, y_batch)
            return loss.mean()
        grad_Ws.append(fd_grad(checker, W.reshape(-1)).reshape(W.shape))
    for i, b in enumerate(biases):
        def checker(b_flat):  # is b already flat?
            biases[i] = b_flat.reshape(b.shape)
            output, layer_caches = forward_pass(X_batch, weight_matrices, biases, activations)            
            loss, dL_dg = logistic_loss(output, y_batch)
            return loss.mean()
        grad_bs.append(fd_grad(checker, b.reshape(-1)).reshape(b.shape))
    return grad_Ws, grad_bs

In [ ]:
layer_dims = [28*28, 20, 1]
activations = [relu_activation, sigmoid_activation]
weight_matrices = create_weight_matrices(layer_dims)
biases = create_bias_vectors(layer_dims)
bs = 100
batch_idx = 1
X_batch = X_train[batch_idx*bs:(batch_idx+1)*bs]
X_batch = X_batch.reshape(X_batch.shape[0], -1)
y_batch = y_train[batch_idx*bs:(batch_idx+1)*bs]



In [ ]:
output, layer_caches = forward_pass(X_batch, weight_matrices, biases, activations)
loss, dL_dg = logistic_loss(output, y_batch)
grad_Ws, grad_bs = backward_pass(dL_dg, layer_caches)


In [ ]:
grad_Ws_fd, grad_bs_fd = backward_pass_checker(X_batch, y_batch, weight_matrices, biases, activations)

for W, W_fd in zip(grad_Ws, grad_Ws_fd):
    print(np.abs(W - W_fd).max())
    print(np.abs(W - W_fd).mean())

for b, b_fd in zip(grad_bs, grad_bs_fd):
    print(np.abs(b - b_fd).max())
    print(np.abs(b - b_fd).mean())


Deliverables

In [ ]:
import pickle
with open("test_batch_weights_biases.pkl", "rb") as fn:
    (X_batch, y_batch, weight_matrices, biases) = pickle.load(fn)
grad_Ws, grad_bs = backward_pass_checker(X_batch, y_batch, weight_matrices, biases, activations)

with np.printoptions(precision=2):
    print(grad_Ws[0])
    print()
    print(grad_Ws[1])
    print()
    print(grad_bs[0])
    print()
    print(grad_bs[1])

In [ ]:
with open("test_batch_weights_biases.pkl", "rb") as fn:
    (X_batch, y_batch, weight_matrices, biases) = pickle.load(fn)
activations = [relu_activation, sigmoid_activation]
output, _ = forward_pass(X_batch, weight_matrices, biases, activations)
loss, dL_dg = logistic_loss(output, y_batch)
print(f"{loss.mean():.4f}")

In [ ]:
with open("test_batch_weights_biases.pkl", "rb") as fn:
    (X_batch, y_batch, weight_matrices, biases) = pickle.load(fn)
activations = [relu_activation, sigmoid_activation]
output, layer_caches = forward_pass(X_batch, weight_matrices, biases, activations)
loss, dL_dg = logistic_loss(output, y_batch)
grad_Ws, grad_bs = backward_pass(dL_dg, layer_caches)

with np.printoptions(precision=2):
    print(grad_Ws[0])
    print()
    print(grad_Ws[1])
    print()
    print(grad_bs[0])
    print()
    print(grad_bs[1])